In [377]:
import os

import numpy as np  # numerical library
import xarray as xr  # netCDF library
import pandas as pd

import matplotlib.pyplot as plt  # plotting library
import cartopy  # Map projections libary
import cartopy.crs as ccrs  # Projections list
import cartopy.feature as cfeature

from shapely.geometry import Polygon, Point
from shapely.ops import transform
import pyproj

import cftime

In [378]:
def point_in_polygon(lon, lat, polygon):
    point = Point(lon, lat)
    return polygon.contains(point)

In [379]:
# Define a transformation to ensure the polygon's CRS matches
def transform_polygon(polygon, src_crs='epsg:4326', tgt_crs='epsg:4326'):
    # Transform the polygon to match the DataArray CRS if needed
    proj = pyproj.Transformer.from_proj(pyproj.Proj(src_crs), pyproj.Proj(tgt_crs), always_xy=True)
    return transform(lambda x, y: proj.transform(x, y), polygon)

In [381]:
Titulos = ['Demarcación marina levantino-balear', 'Demarcación marina noratlántica','Demarcación marina canaria', 
           'Demarcación sudatlántica','Demarcación Estrecho y Alborán']
Titulos_short = ['LEB', 'NOR','CAN','SUD','ESA']

In [382]:
HOME=os.environ['HOME']   
f = open(HOME+'/.env', 'r')
for line in f.readlines():
    Name=line.strip().split('=')[0]
    Content=line.strip().split('=')[-1]
    if Name=='dirData' or Name=='dirAnalisis':
        exec(Name + "=" + "'" + Content + "'")
f.close()
base_file1 = dirData + '/Climatologias/Glorys/cmems_mod_glo_phy_my_0.083_P1M-m_202311/'
base_file2 = dirData + '/Climatologias/Glorys/cmems_mod_glo_phy_myint_0.083deg_P1M-m_202311/'
dataDir    = dirAnalisis + '/IEOOSWebPage/data'
imageDir   = dirAnalisis + '/IEOOSWebPage/images'

In [384]:
i=2
titulo = Titulos[i]
titulo_short = Titulos_short[i]

In [385]:
# Load the data from the .txt file
demCoord = []
longDem, latiDem = [], []
with open('./LimiteDemarcaciones/Demarcacion'+titulo_short+'.txt', 'r') as f:
    for line in f:
        # Split the line by whitespace and append the values
        longitude, latitude = map(float, line.split())
        longDem.append(longitude)
        latiDem.append(latitude)
        demCoord.append((longitude,latitude))
demPolygon = Polygon(demCoord)    
demPolygon_transformed = transform_polygon(demPolygon)

## Read the data

In [386]:
files = []

for iy in range(1995,2021):
    for im in range(1,13):
        files.append(base_file1+"mercatorglorys12v1_gl12_mean_%04d%02d.nc"%(iy,im))

for iy in range(2021,2022):
    for im in range(1,7):
        files.append(base_file1+"mercatorglorys12v1_gl12_mean_%04d%02d.nc"%(iy,im))

for iy in range(2021,2022):
    for im in range(7,13):
        files.append(base_file2+"mercatorglorys12v1_gl12_mean_%04d%02d.nc"%(iy,im))

for iy in range(2022,2024):
    for im in range(1,13):
        files.append(base_file2+"mercatorglorys12v1_gl12_mean_%04d%02d.nc"%(iy,im))

In [395]:
len(files)

348

In [387]:
DC = xr.open_mfdataset(files)

In [388]:
DC = DC.drop_vars("mlotst").drop_vars("zos")
DC = DC.drop_vars("sithick").drop_vars("siconc")
DC = DC.drop_vars("usi").drop_vars("vsi").drop_vars("uo").drop_vars("vo").drop_vars("bottomT")

In [389]:
if titulo_short == 'LEB':
        slicelatitude=slice(35.5,42.75)
        slicelongitude=slice(358-360,368-360)
        DC_temp=DC.thetao.sel(latitude=slicelatitude).sel(longitude=slicelongitude).load()
        DC_salt=DC.so.sel(latitude=slicelatitude).sel(longitude=slicelongitude).load()
        print('>>>>> '+titulo)        

elif  titulo_short == 'NOR':
        slicelatitude=slice(41.5,46.9)
        slicelongitude=slice(346-360,-1.5)
        DC_temp=DC.thetao.sel(latitude=slicelatitude).sel(longitude=slicelongitude).load()
        DC_salt=DC.so.sel(latitude=slicelatitude).sel(longitude=slicelongitude).load()
        print('>>>>> '+titulo)        
        
elif  titulo_short == 'CAN':
        slicelatitude=slice(24,32.5)
        slicelongitude=slice(335-360,350-360)
        DC_temp=DC.thetao.sel(latitude=slicelatitude).sel(longitude=slicelongitude).load()
        DC_salt=DC.so.sel(latitude=slicelatitude).sel(longitude=slicelongitude).load()
        
        mask = np.array([[point_in_polygon(lon,lat,demPolygon_transformed) 
         for lon in DC_temp.longitude.values] 
         for lat in DC_temp.latitude.values])
        
        DC_temp_umasked = DC_temp
        DC_salt_umasked = DC_salt

        DC_temp = DC_temp.where(mask)
        DC_salt = DC_salt.where(mask)
        
        print('>>>>> '+titulo)

elif  titulo_short == 'SUD':
        slicelatitude=slice(35.5,37.5)
        slicelongitude=slice(352-360,354.5-360)
        DC_temp=DC.thetao.sel(latitude=slicelatitude).sel(longitude=slicelongitude).load()
        DC_salt=DC.so.sel(latitude=slicelatitude).sel(longitude=slicelongitude).load()
        print('>>>>> '+titulo)

elif  titulo_short == 'ESA':
        slicelatitude=slice(35.5,37)
        slicelongitude=slice(354-360,358.5-360)
        DC_temp=DC.thetao.sel(latitude=slicelatitude).sel(longitude=slicelongitude).load()
        DC_salt=DC.so.sel(latitude=slicelatitude).sel(longitude=slicelongitude).load()

print('>>>>> '+titulo)

>>>>> Demarcación marina canaria
>>>>> Demarcación marina canaria


In [390]:
DC_temp.to_netcdf('./Data/Glorys_DC_temp'+titulo_short+'.nc')

In [391]:
DC_temp

<xarray.DataArray 'thetao' (time: 348, depth: 50, latitude: 103, longitude: 181)> Size: 3GB
array([[[[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
...
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]]]])
Coordinates:
  * longitude  (longitude) float32 724B -25.0 -24.92 -24.83 ... -10.08 -10.0
  * latitude   (latitude) float32 412B 24.0 24.08 24.17 ... 32.33 32.42 32.5
  * depth      (depth) float32 200B 0.494 1.541 2.646 ... 5.275e+03 5.728e+03
  * time       (time) datetime64[ns] 3kB 1995-01-16T12:00:00 ... 2023-12-16T1...
Attributes:
    long_name:      Temperature
    standard_name:  sea_water_potential_temperature
    units:          degrees_C
    unit_long:      Degrees Celsius
    valid_min:      -32748
    valid_max:      20030
    cell_methods:   area: mean